## 🥈 Silver Layer

Tratamento dos dados

In [0]:
%%capture

%run ./00-setup

In [0]:
# Lê a base bronze
df_bronze = spark.read.table("sandbox_prd.bronze_layer.streaming_history_user_spotify")

# Listas colunas para remover 
drop_columns = [
    "audiobook_chapter_title",
    "audiobook_chapter_uri",
    "audiobook_title",
    "audiobook_uri",
    "conn_country",
    "incognito_mode",
    "ip_addr",
    "dt_ingestion",
    "source_file"
]

# Remove as colunas desnecessárias
df_silver = df_bronze.drop(*drop_columns)

In [0]:
v_hora_brasil = hour(from_utc_timestamp(col("ts"), "America/Sao_Paulo"))

df_silver = df_silver.select(
    # Renomia e alteara o tipo da coluna
    col("episode_name").alias("nm_episode_name"),
    col("episode_show_name").alias("nm_episode_show_name"),
    col("master_metadata_album_album_name").alias("nm_album_name"),
    col("master_metadata_album_artist_name").alias("nm_artist_name"),
    col("master_metadata_track_name").alias("nm_track_name"),
    col("ms_played").alias("qt_played_ms"),
    col("offline").alias("fl_offline"),
    col("offline_timestamp").alias("ts_offline"),
    col("platform").alias("ds_platform"),
    col("reason_end").alias("ds_reason_end"),
    col("reason_start").alias("ds_reason_start"),
    col("shuffle").alias("fl_shuffle"),
    col("skipped").alias("fl_skipped"),
    col("ts").cast("Timestamp").alias("ts_streaming"),
    col("spotify_episode_uri").alias("ds_spotify_episode_uri"),
    col("spotify_track_uri").alias("ds_spotify_track_uri"),
    v_hora_brasil.alias("nr_hora_brasil"),

    # Cria coluna ano mês
    concat(
        year(col("ts")).cast("String"),
        lit("-"),
        month(col("ts")).cast("String")
    ).alias("dt_ano_mes"),

    # Cria coluna duração em segundos
    (col("ms_played")/1000).cast("Int").alias("ts_duration_seconds"),

    # Cria coluna de minutos
    (col("ms_played")/1000/60).cast("Int").alias("ts_duration_minutes"),

    # Cria coluna de dia da semana
    when(dayofweek(col("ts")) == 1, "Domingo")
        .when(dayofweek(col("ts")) == 2, "Segunda-feira")
        .when(dayofweek(col("ts")) == 3, "Terça-feira")
        .when(dayofweek(col("ts")) == 4, "Quarta-feira")
        .when(dayofweek(col("ts")) == 5, "Quinta-feira")
        .when(dayofweek(col("ts")) == 6, "Sexta-feira")
        .when(dayofweek(col("ts")) == 7, "Sábado")
        .alias("ds_day_of_week"),
    
    # Cria coluna período do dia
    when(v_hora_brasil < 6, "Madrugada")
        .when(v_hora_brasil < 12, "Manhã")
        .when(v_hora_brasil < 18, "Tarde")
        .otherwise("Noite").alias("ds_periodo_dia"),

    # Cria coluna ordem do período do dia
    when(v_hora_brasil < 6, 1)
    .when(v_hora_brasil < 12, 2)
    .when(v_hora_brasil < 18, 3)
    .otherwise(4).alias("nr_ordem_periodo"),


    # Cria coluna tipo do inicio
    when(col("reason_start") == "trackdone", "Reprodução Automática")
    .when(col("reason_start") == "clickrow", "Seleção Manual")
    .when(col("reason_start") == "appload", "Retomada App")
    .when(col("reason_start") == "playbtn", "Botão Play")
    .when(col("reason_start").isin("fwdbtn", "backbtn"), "Navegação (Pular/Voltar)")
    .when(col("reason_start") == "remote", "Controle Externo")
    .otherwise("Outros").alias("ds_tipo_inicio"),

    # Cria coluna device_type
    when(lower(col("platform")).contains("android"), "Android")
    .when(lower(col("platform")).contains("ios"), "iOS")
    .when(lower(col("platform")).contains("web"), "Web")
    .when(lower(col("platform")).contains("windows"), "Windows")
    .when(lower(col("platform")).contains("mac"), "Mac")
    .when(lower(col("platform")).contains("linux"), "Linux")
    .when(lower(col("platform")).contains("tv"), "TV")
    .when(lower(col("platform")).contains("echo_show_5"), "Echo_Show")
    .when(lower(col("platform")).contains("other"), "Outros")
    .otherwise("Não identificado").alias("ds_device_type"),


    # Cria coluna de link de musica clicavel
   when(col("master_metadata_track_name").isNotNull(), 
         concat(lit("https://open.spotify.com/track/"), col("spotify_track_uri"))
    )
    .when(col("master_metadata_album_album_name").isNotNull(), 
          concat(lit("https://open.spotify.com/album/"), col("spotify_track_uri"))
    )
    .when(col("episode_show_name").isNotNull(), 
          concat(lit("https://open.spotify.com/episode/"), col("spotify_track_uri"))
    )
    .otherwise(lit("Não identificado")).alias("ds_link_musica"),
    to_date(col("ts")).alias("dt_referencia"),

)


In [0]:
(df_silver.write
    .format("delta")
    .mode("overwrite")              
    .option("mergeSchema", "true")  
    .saveAsTable(NOME_TABELA_DESTINO_SILVER)
)

print(f"✅ Carga concluída com sucesso em: {NOME_TABELA_DESTINO_SILVER}")